# xgboost 

2017-05-21

In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date,time

### 录入数据 

In [3]:
travel_time_train_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/final_data/加工好的数据/6.0/travel_time_train_data.csv')
volume_train_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/final_data/加工好的数据/6.0/volume_train_data.csv')
travel_time_test_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/final_data/加工好的数据/6.5/test_travel_time_data.csv')
volume_test_data = pd.read_csv('/home/godcedric/GitLocal/KDDCUP2017/final_data/加工好的数据/6.5/test_volume_data.csv')

### 准备数据 

In [4]:
# 分割训练集和验证集
test_date = ['2016-10-18', '2016-10-19', '2016-10-20', '2016-10-21', '2016-10-22', '2016-10-23', '2016-10-24']
def isin(x):
    if x in test_date:
        return 1
    else:
        return 0
# 平均时间
travel_time_train_data['is_test'] = travel_time_train_data['date'].map(isin)
travel_time_train = travel_time_train_data[travel_time_train_data['is_test'] == 0]
travel_time_val = travel_time_train_data[travel_time_train_data['is_test'] == 1]
# 流量
volume_train_data['is_test'] = volume_train_data['date'].map(isin)
volume_train = volume_train_data[volume_train_data['is_test'] == 0]
volume_val = volume_train_data[volume_train_data['is_test'] == 1]

In [5]:
# 分离标签和特征
Y_time = travel_time_train['avg_travel_time']
X_time = travel_time_train.drop(['avg_travel_time'], axis=1)
val_Y_time = travel_time_val.avg_travel_time
val_X_time = travel_time_val.drop(['avg_travel_time'], axis=1)


Y_volume = volume_train['volume']
X_volume = volume_train.drop(['volume'], axis=1)
val_Y_volume = volume_val.volume
val_X_volume = volume_val.drop(['volume'], axis=1)

In [6]:
# 选择特征
travel_time_features = ['route', 'weekday', 'timemap',\
                        'pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation',\
                        'last_20min',\
                        'SSD',\
                        'is_workday'  
                       ]

volume_features = ['pair', 'weekday', 'timemap',\
                   'pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation',\
                   'last_20min',\
                   'SSD',\
                   'is_workday'  
                  ]

In [73]:
# 选择特征
travel_time_features = ['A-2', 'A-3', 'B-1', 'B-3', 'C-1', 'C-3',
                        'weekday__0', 'weekday__1', 'weekday__2', 'weekday__3', 'weekday__4', 'weekday__5', 'weekday__6', 
                        'workday__1', 'workday__2', 'workday__3',
                        'hour__0', 'hour__1', 'hour__2','hour__3', 'hour__4', 'hour__5', 'hour__6', 'hour__7', 'hour__8',
                        'hour__9', 'hour__10', 'hour__11', 'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16', 
                        'hour__17', 'hour__18', 'hour__19', 'hour__20',
                        'hour__21', 'hour__22', 'hour__23', 'minute__0', 'minute__20', 'minute__40',
                        'pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation',\
                        #'last_20min',\
                        'SSD'
                       ]

volume_features = ['1-0', '1-1', '2-0', '3-0', '3-1',
                   'weekday__1', 'weekday__2', 'weekday__3', 'weekday__4', 'weekday__5', 'weekday__6', 'workday__1', 'workday__2',
                   'hour__0', 'hour__1', 'hour__2','hour__3', 'hour__4', 'hour__5', 'hour__6', 'hour__7', 'hour__8',
                   'hour__9', 'hour__10', 'hour__11', 'hour__12', 'hour__13', 'hour__14', 'hour__15', 'hour__16', 
                   'hour__17', 'hour__18', 'hour__19', 'hour__20',
                   'hour__21', 'hour__22', 'hour__23', 'minute__0', 'minute__20', 'minute__40',
                   'pressure', 'wind_direction', 'wind_speed', 'temperature', 'rel_humidity', 'precipitation',\
                   #'last_20min',\
                   'SSD'
                  ]

In [7]:
X_time = X_time[travel_time_features]
val_X_time = val_X_time[travel_time_features]
travel_time_submission = travel_time_test_data[['intersection_id', 'tollgate_id', 'time_window']]
travel_time_test_data = travel_time_test_data[travel_time_features]

X_volume = X_volume[volume_features]
val_X_volume = val_X_volume[volume_features]
volume_submission = volume_test_data[['tollgate_id', 'time_window', 'direction']]
volume_test_data = volume_test_data[volume_features]

In [8]:
# 字符串特征转换数值
X_time['route'] = pd.factorize(X_time['route'])[0]
val_X_time['route'] = pd.factorize(val_X_time['route'])[0]
travel_time_test_data['route'] = pd.factorize(travel_time_test_data['route'])[0]
X_volume['pair'] = pd.factorize(X_volume['pair'])[0]
val_X_volume['pair'] = pd.factorize(val_X_volume['pair'])[0]
volume_test_data['pair'] = pd.factorize(volume_test_data['pair'])[0]

In [9]:
# 转化格式
time_train = xgb.DMatrix(X_time, label=Y_time)
time_val = xgb.DMatrix(val_X_time, label=val_Y_time)
time_test = xgb.DMatrix(travel_time_test_data)

volume_train = xgb.DMatrix(X_volume, label=Y_volume)
volume_val = xgb.DMatrix(val_X_volume, label=val_Y_volume)
volume_test = xgb.DMatrix(volume_test_data)

### 参数设置 

In [31]:
params = {
    'booster':'gbtree',      #gbtree和gblinear
    'objective':'reg:linear',
    'eta':0.04,             #为了防止过拟合，更新过程中用到的收缩步长。在每次提升计算之后，算法会直接获得新特征的权重。eta通过缩减特征的权重使提升计算过程更加保守。缺省值为0.3,通常最后设置eta为0.01~0.2
    #'gamma':0.3,            #模型在默认情况下，对于一个节点的划分只有在其loss function得到结果大于0的情况下才进行，而gamma 给定了所需的最低loss function的值gamma值使得算法更conservation，且其值依赖于loss function ，
    'max_depth':4,          #树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合,建议通过交叉验证（xgb.cv ) 进行调参,通常取值：3-10
    #'min_child_weight':2,   #孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。该成熟越大算法越conservative。即调大这个参数能够控制过拟合。
    #'max_delta_step':2,     #如果取值为0，那么意味着无限制。如果取为正数，则其使得xgboost更新过程更加保守。
    'subsample':1,          #用于训练模型的子样本占整个样本集合的比例。如果设置为0.5则意味着XGBoost将随机的从整个样本集合中抽取出50%的子样本建立树模型，这能够防止过拟合。
    'colsample_bytree':1,   #在建立树时对特征随机采样的比例。缺省值为1 
    #'alpha':5,              #L1 正则的惩罚系数,当数据维度极高时可以使用，使得算法运行更快。
    #'lambda':40,           #L2 正则的惩罚系数,用于处理XGBoost的正则化部分。通常不使用，但可以用来降低过拟合
    'seed':2017,
}

### 平均时间训练

In [32]:
# 自定义评价函数
def MAPE(preds, dtrain):
    labels = dtrain.get_label()
    return 'MAPE', float(sum(np.fabs((labels - preds) / labels))) / len(labels)
plst = list(params.items())
num_rounds = 1000
watchlist = [(time_train, 'train'), (time_val, 'val')]

time_model = xgb.train(plst, time_train, num_rounds, watchlist, feval=MAPE)
print("best best_ntree_limit", time_model.best_ntree_limit)

[0]	train-rmse:125.295	val-rmse:127.97	train-MAPE:0.952075	val-MAPE:0.953105
[1]	train-rmse:120.673	val-rmse:123.314	train-MAPE:0.911065	val-MAPE:0.913069
[2]	train-rmse:116.249	val-rmse:118.894	train-MAPE:0.871708	val-MAPE:0.874316
[3]	train-rmse:112.016	val-rmse:114.624	train-MAPE:0.834088	val-MAPE:0.83743
[4]	train-rmse:107.965	val-rmse:110.532	train-MAPE:0.798072	val-MAPE:0.801876
[5]	train-rmse:104.091	val-rmse:106.599	train-MAPE:0.763652	val-MAPE:0.767756
[6]	train-rmse:100.386	val-rmse:102.893	train-MAPE:0.730727	val-MAPE:0.735422
[7]	train-rmse:96.8415	val-rmse:99.2786	train-MAPE:0.699247	val-MAPE:0.704317
[8]	train-rmse:93.457	val-rmse:95.8532	train-MAPE:0.669087	val-MAPE:0.674483
[9]	train-rmse:90.224	val-rmse:92.5777	train-MAPE:0.640185	val-MAPE:0.645659
[10]	train-rmse:87.1351	val-rmse:89.4281	train-MAPE:0.612536	val-MAPE:0.618123
[11]	train-rmse:84.1837	val-rmse:86.4049	train-MAPE:0.586124	val-MAPE:0.591603
[12]	train-rmse:81.37	val-rmse:83.5835	train-MAPE:0.560823	val-MAP

In [13]:
time_model.get_fscore()

{'SSD': 35,
 'is_workday': 50,
 'last_20min': 238,
 'precipitation': 13,
 'pressure': 51,
 'rel_humidity': 61,
 'route': 84,
 'temperature': 43,
 'timemap': 190,
 'weekday': 21,
 'wind_direction': 42,
 'wind_speed': 49}

### 预测输出 

In [14]:
time_predict = time_model.predict(time_test, ntree_limit=time_model.best_ntree_limit)

In [87]:
travel_time_submission['avg_travel_time'] = time_predict
travel_time_submission.to_csv('/home/godcedric/GitLocal/KDDCUP2017/submission_result/phase1.5/xgboost/独热结果/travel_time_submission.csv', index=False)

### 流量训练

In [19]:
params = {
    'booster':'gbtree',      #gbtree和gblinear
    'objective':'reg:linear',
    'eta':0.045,             #为了防止过拟合，更新过程中用到的收缩步长。在每次提升计算之后，算法会直接获得新特征的权重。eta通过缩减特征的权重使提升计算过程更加保守。缺省值为0.3,通常最后设置eta为0.01~0.2
    #'gamma':0.3,            #模型在默认情况下，对于一个节点的划分只有在其loss function得到结果大于0的情况下才进行，而gamma给定了所需的最低loss function的值gamma值使得算法更conservation，且其值依赖于loss function ，
    'max_depth':5,          #树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合,建议通过交叉验证（xgb.cv ) 进行调参,通常取值：3-10
    #'min_child_weight':0,   #孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。该成熟越大算法越conservative。即调大这个参数能够控制过拟合。
    #'max_delta_step':0,     #如果取值为0，那么意味着无限制。如果取为正数，则其使得xgboost更新过程更加保守。
    'subsample':0.8,          #用于训练模型的子样本占整个样本集合的比例。如果设置为0.5则意味着XGBoost将随机的从整个样本集合中抽取出50%的子样本建立树模型，这能够防止过拟合。
    'colsample_bytree':1,   #在建立树时对特征随机采样的比例。缺省值为1 
    #'alpha':0,              #L1 正则的惩罚系数,当数据维度极高时可以使用，使得算法运行更快。
    #'lambda':120,           #L2 正则的惩罚系数,用于处理XGBoost的正则化部分。通常不使用，但可以用来降低过拟合
    'seed':2016,
}

In [20]:
plst = list(params.items())
num_rounds = 1000
watchlist = [(volume_train, 'train'), (volume_val, 'val')]

volume_model = xgb.train(plst, volume_train, num_rounds, watchlist, feval=MAPE)
print("best best_ntree_limit", volume_model.best_ntree_limit )

[0]	train-rmse:64.6852	val-rmse:62.4729	train-MAPE:0.917806	val-MAPE:0.917581
[1]	train-rmse:61.9323	val-rmse:60.0398	train-MAPE:0.869758	val-MAPE:0.871551
[2]	train-rmse:59.2955	val-rmse:57.4881	train-MAPE:0.828879	val-MAPE:0.830108
[3]	train-rmse:56.7847	val-rmse:55.0509	train-MAPE:0.790873	val-MAPE:0.791193
[4]	train-rmse:54.3819	val-rmse:52.726	train-MAPE:0.755529	val-MAPE:0.75533
[5]	train-rmse:52.094	val-rmse:50.512	train-MAPE:0.722354	val-MAPE:0.721594
[6]	train-rmse:49.9158	val-rmse:48.4086	train-MAPE:0.692864	val-MAPE:0.690593
[7]	train-rmse:47.8373	val-rmse:46.404	train-MAPE:0.665169	val-MAPE:0.661431
[8]	train-rmse:45.8517	val-rmse:44.483	train-MAPE:0.638834	val-MAPE:0.634719
[9]	train-rmse:43.9604	val-rmse:42.6615	train-MAPE:0.614782	val-MAPE:0.609804
[10]	train-rmse:42.1611	val-rmse:40.9138	train-MAPE:0.591792	val-MAPE:0.585563
[11]	train-rmse:40.4348	val-rmse:39.2453	train-MAPE:0.570541	val-MAPE:0.563267
[12]	train-rmse:38.806	val-rmse:37.6619	train-MAPE:0.551368	val-MAPE

In [92]:
volume_model.get_fscore()

{'1-0': 1324,
 '1-1': 1972,
 '2-0': 1164,
 '3-0': 1521,
 '3-1': 1561,
 'SSD': 771,
 'hour__0': 730,
 'hour__1': 59,
 'hour__10': 369,
 'hour__11': 789,
 'hour__12': 292,
 'hour__13': 613,
 'hour__14': 586,
 'hour__15': 329,
 'hour__16': 477,
 'hour__17': 575,
 'hour__18': 397,
 'hour__19': 237,
 'hour__2': 49,
 'hour__20': 199,
 'hour__21': 142,
 'hour__22': 78,
 'hour__23': 102,
 'hour__3': 50,
 'hour__4': 57,
 'hour__5': 160,
 'hour__6': 318,
 'hour__7': 521,
 'hour__8': 471,
 'hour__9': 720,
 'minute__0': 1580,
 'minute__20': 1060,
 'minute__40': 1509,
 'pressure': 1059,
 'rel_humidity': 729,
 'temperature': 893,
 'weekday__5': 577,
 'weekday__6': 595,
 'wind_direction': 1248,
 'wind_speed': 1203,
 'workday__1': 827}

### 预测输出

In [93]:
volume_predict = volume_model.predict(volume_test, ntree_limit=volume_model.best_ntree_limit)

In [94]:
volume_submission['volume'] = volume_predict
volume_submission.to_csv('/home/godcedric/GitLocal/KDDCUP2017/submission_result/phase1.5/xgboost/独热结果/volume_submission.csv', index=False)